In [1]:
# Program for learning the performance of 6 layer deep neural network inverted pyramid strucuture

import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.datasets import cifar10
from matplotlib import pyplot 
from scipy.misc import toimage
import datetime
from keras.layers import Activation, Dense

K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

#dataset image parameters
img_rows=32
img_cols=32
channel=3
nb_classes=10

In [3]:
(x_train, y_train),(x_test,y_test)=cifar10.load_data()

x_train = x_train.reshape(x_train.shape[0], channel,img_rows, img_cols)
x_test = x_test.reshape(x_test.shape[0], channel,img_rows, img_cols)
input_shape=(3,img_rows,img_cols)

In [4]:
x_test=x_test.astype('float32')
x_train=x_train.astype('float32')
x_test=x_test/255.
x_train=x_train/255.

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]


In [6]:
# Defining model
vgg = Sequential()
vgg.add(Convolution2D(40, 2, 2, input_shape=input_shape, border_mode='valid'))
vgg.add(Activation('relu'))
#vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Convolution2D(30, 2, 2, border_mode='valid'))
vgg.add(Activation('relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Convolution2D(20, 2, 2, border_mode='valid'))
vgg.add(Activation('relu'))
#vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Convolution2D(15, 2, 2, border_mode='valid'))
vgg.add(Activation('relu'))
vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Convolution2D(10, 2, 2, border_mode='valid'))
vgg.add(Activation('relu'))
#vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Convolution2D(5, 2, 2, border_mode='valid'))
vgg.add(Activation('relu'))
#vgg.add(MaxPooling2D((2, 2), strides=(2, 2), border_mode='valid'))

vgg.add(Flatten())
vgg.add(Dense(200))
vgg.add(Dropout(0.5))
vgg.add(Activation('relu'))
vgg.add(Dense(nb_classes, activation='softmax'))

model = vgg
print (model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 40, 31, 31)    520         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 40, 31, 31)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 30, 30, 30)    4830        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 30, 30, 30)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [7]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 40, 31, 31)    520         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 40, 31, 31)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 30, 30, 30)    4830        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 30, 30, 30)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [8]:
# printing the time when the training starts
print(datetime.datetime.now())

# Fit the model
model.fit(x_train, y_train, validation_split=0.2, nb_epoch=epochs, batch_size=32)

# printing the time when the training finishes
print(datetime.datetime.now())


2017-04-20 19:11:28.142634
Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 164s - loss: 2.0774 - acc: 0.2167 - val_loss: 1.8256 - val_acc: 0.3183
Epoch 2/25
40000/40000 [==============================] - 167s - loss: 1.8146 - acc: 0.3358 - val_loss: 1.6577 - val_acc: 0.3862
Epoch 3/25
40000/40000 [==============================] - 168s - loss: 1.6744 - acc: 0.3840 - val_loss: 1.5875 - val_acc: 0.4170
Epoch 4/25
40000/40000 [==============================] - 167s - loss: 1.5406 - acc: 0.4328 - val_loss: 1.4163 - val_acc: 0.4815
Epoch 5/25
40000/40000 [==============================] - 168s - loss: 1.4575 - acc: 0.4671 - val_loss: 1.4284 - val_acc: 0.4922
Epoch 6/25
40000/40000 [==============================] - 168s - loss: 1.3983 - acc: 0.4937 - val_loss: 1.3178 - val_acc: 0.5241
Epoch 7/25
40000/40000 [==============================] - 168s - loss: 1.3512 - acc: 0.5087 - val_loss: 1.3020 - val_acc: 0.5251
Epoch 8/25
40000/400

In [9]:
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 60.40%
